<a href="https://colab.research.google.com/github/roitraining/SparkforDataEngineers/blob/Development/Ch03_SparkSQL_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

rootpath = '/home/student/ROI/Spark/'
datapath = f'{rootpath}datasets/'
sys.path.append(rootpath)
from pyspark_helpers import *
sc, spark, conf = initspark()


initializing pyspark
pyspark initialized


### Homework ##

### First Challenge
Create a Python function to determine if a number is odd or even and use that to select only the even numbered shippers from the TSV folder of northwind. Note the TSV file does not have headers so you will need to do something to make the DataFrame have a meaningful structure. I would suggest using SparkSql as much as possible to rename and cast the columns which are ShipperID, CompanyName, and Phone.

### Second Challenge
Take the Order_LineItems.json folder, read it into a DataFrame, and flatten it and then calculate the average price paid for a product.


In [ ]:
o = spark.read.csv(f'{datapath}/northwind/CSVHeaders/orders', header = True, inferSchema = True)
od = spark.read.csv(f'{datapath}/northwind/CSVHeaders/orderdetails', header = True, inferSchema = True)

o.createOrReplaceTempView('Orders')
od.createOrReplaceTempView('OrderDetails')
sql = """
select o.OrderID, o.CustomerID, o.OrderDate
           , COLLECT_SET(NAMED_STRUCT("ProductID", od.ProductID, 
                                      "UnitPrice", od.UnitPrice,
                                      "Quantity", od.Quantity,
                                      "Discount", od.discount)) as LineItems
from Orders as o join OrderDetails as od on o.OrderID = od.OrderID
GROUP BY o.OrderID, o.CustomerID, o.OrderDate
ORDER BY o.OrderID
"""
od2 = spark.sql(sql)
od2.write.mode('overwrite').json(f'{rootpath}Orders_LineItems.json')
print('Done')



In [ ]:
shippers = spark.read.csv(f'{datapath}/northwind/TSV/shippers', header=False, sep = '\t')
#shippers.show()
print(shippers)
shippers.createOrReplaceTempView('shippers')
print(shippers.collect())
shippers = spark.sql('select cast(_c0 as int) as ShipperID, _c1 as ShipperName, _c2 as Phone from shippers')
shippers.createOrReplaceTempView('shippers')
print(shippers)

def isEven(x):
    return x % 2 == 0

from pyspark.sql.functions import udf
from pyspark.sql.types import *

spark.udf.register('isEven', isEven, BooleanType())

spark.sql('select * FROM Shippers WHERE isEven(ShipperID)').show()


In [2]:
o = spark.read.json(f'{rootpath}/Orders_LineItems.json')
display(o)
o.createOrReplaceTempView('Orders')
sql = '''
select OrderId, CustomerID, OrderDate, l.ProductID, l.UnitPrice, l.Quantity, l.Discount
FROM Orders LATERAL VIEW EXPLODE(LineItems) EXPLODED_TABLE AS l
'''
o2 = spark.sql(sql)
display(o2)
o2.createOrReplaceTempView('FlatOrders')
spark.sql('select avg(UnitPrice) from FlatOrders').show()


,CustomerID,LineItems,OrderDate,OrderID,OrderTotal,cnt
0,SIMOB,"[(0.05, 16, 14, 17.45)]",1998-05-06T00:00:00.000Z,11074,244.3,1
1,RICSU,"[(0.15, 76, 2, 18.0), (0.15, 2, 10, 19.0), (0....",1998-05-06T00:00:00.000Z,11075,586.0,3
2,BONAP,"[(0.25, 19, 10, 9.2), (0.25, 14, 20, 23.25), (...",1998-05-06T00:00:00.000Z,11076,1057.0,3
3,RATTC,"[(0.03, 16, 2, 17.45), (0.1, 8, 2, 40.0), (0.2...",1998-05-06T00:00:00.000Z,11077,1374.6,25
4,FURIB,"[(0.15, 16, 40, 17.45), (0.15, 35, 20, 18.0), ...",1997-05-28T00:00:00.000Z,10551,1836.0,3
5,HILAA,"[(0.0, 69, 18, 36.0), (0.0, 75, 30, 7.75)]",1997-05-29T00:00:00.000Z,10552,880.5,2
6,WARTH,"[(0.0, 31, 30, 12.5), (0.0, 16, 14, 17.45), (0...",1997-05-30T00:00:00.000Z,10553,1546.3,5
7,OTTIK,"[(0.05, 23, 20, 9.0), (0.05, 77, 10, 13.0), (0...",1997-05-30T00:00:00.000Z,10554,1819.5,4
8,SAVEA,"[(0.2, 56, 40, 38.0), (0.2, 24, 18, 4.5), (0.2...",1997-06-02T00:00:00.000Z,10555,3680.5,5
9,ISLAT,"[(0.0, 2, 10, 19.0), (0.0, 13, 10, 6.0), (0.0,...",1998-01-13T00:00:00.000Z,10829,1764.0,4


,OrderId,CustomerID,OrderDate,ProductID,UnitPrice,Quantity,Discount
0,11074,SIMOB,1998-05-06T00:00:00.000Z,16,17.45,14,0.05
1,11075,RICSU,1998-05-06T00:00:00.000Z,76,18.00,2,0.15
2,11075,RICSU,1998-05-06T00:00:00.000Z,2,19.00,10,0.15
3,11075,RICSU,1998-05-06T00:00:00.000Z,46,12.00,30,0.15
4,11076,BONAP,1998-05-06T00:00:00.000Z,19,9.20,10,0.25
5,11076,BONAP,1998-05-06T00:00:00.000Z,14,23.25,20,0.25
6,11076,BONAP,1998-05-06T00:00:00.000Z,6,25.00,20,0.25
7,11077,RATTC,1998-05-06T00:00:00.000Z,16,17.45,2,0.03
8,11077,RATTC,1998-05-06T00:00:00.000Z,8,40.00,2,0.10
9,11077,RATTC,1998-05-06T00:00:00.000Z,2,19.00,24,0.20


+------------------+
|    avg(UnitPrice)|
+------------------+
|26.218519721577714|
+------------------+

